## Suffering niggas

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh.plotting
import catboost
import sklearn.preprocessing, sklearn.feature_selection, sklearn.model_selection
from sklearn.pipeline import Pipeline, make_pipeline
import sklearn.base
from sklearn.base import BaseEstimator, TransformerMixin

SEED = 42
np.random.seed = SEED

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
display(
    X_train.shape,
    X_train.columns,
    X_train.isnull().sum(),
)

(59400, 40)

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [4]:
pd.Series({c: ([X_train[c].dtype] + list(X_train[c].unique()[:4])).__repr__() for c in X_train.columns})

amount_tsh                     [dtype('float64'), 6000.0, 0.0, 25.0, 20.0]
basin                    [dtype('O'), 'Lake Nyasa', 'Lake Victoria', 'P...
construction_year                 [dtype('int64'), 1999, 2010, 2009, 1986]
date_recorded            [dtype('O'), '2011-03-14', '2013-03-06', '2013...
district_code                                [dtype('int64'), 5, 2, 4, 63]
extraction_type          [dtype('O'), 'gravity', 'submersible', 'swn 80...
extraction_type_class    [dtype('O'), 'gravity', 'submersible', 'handpu...
extraction_type_group    [dtype('O'), 'gravity', 'submersible', 'swn 80...
funder                   [dtype('O'), 'Roman', 'Grumeti', 'Lottery Club...
gps_height                          [dtype('int64'), 1390, 1399, 686, 263]
id                             [dtype('int64'), 69572, 8776, 34310, 67743]
installer                [dtype('O'), 'Roman', 'GRUMETI', 'World vision...
latitude                 [dtype('float64'), -9.8563217699999992, -2.147...
lga                      

In [5]:
y_train.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [6]:
import dateutil.parser

In [7]:
class HideMissingValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for c in X.columns:
            if np.issubdtype(X[c].dtype, np.number):
                X.loc[:,c].fillna(X.loc[:,c].median(), inplace=True)
            else:
                X.loc[:,c].fillna('Unknown', inplace=True)
                X.loc[:,c] = X[c].map(lambda x: str(x))
        return X
    
    
class DropGarbage(BaseEstimator, TransformerMixin):
    def __init__(self, cols=['id', 'funder', 'recorded_by', 'date_recorded']):
        self.cols = cols
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.cols, axis=1)

class SplitDate(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        def t(x):
            if isinstance(x, str):
                x = dateutil.parser.parse(x)
            return (x.year, x.month, x.day, x.weekday())
        X = X.copy()
        (X.loc[:, self.col + '_year'],
         X.loc[:, self.col + '_month'],
         X.loc[:, self.col + '_day'],
         X.loc[:, self.col + '_weekday']) = zip(
        *X.loc[:, self.col].map(t))
        return X

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
prepr1 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('drop', DropGarbage(cols=['id', 'recorded_by', ]))
])

y_enc = sklearn.preprocessing.LabelEncoder()
X = prepr1.fit_transform(X_train)
y = y_enc.fit_transform(y_train.iloc[:,1])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.66)

!date
cbc1 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_te, y_te))
cbc1.fit(X_tr,
         y_tr,
         cat_features=np.where(np.array([dt == object for dt in X.dtypes]))[0],
         verbose=False)
!date

Fri Jul 28 01:42:21 MSK 2017
Fri Jul 28 01:47:03 MSK 2017


In [10]:
cbc1.score(X_te, y_te)

0.80550604080015842

In [11]:
X_train.iloc[0,[1,2]].values

array([6000.0, '2011-03-14'], dtype=object)

In [12]:
!head SubmissionFormat.csv

id,status_group
50785,predicted label
51630,predicted label
17168,predicted label
45559,predicted label
49871,predicted label
52449,predicted label
24806,predicted label
28965,predicted label
36301,predicted label


In [13]:
y_test1 = cbc1.predict(prepr1.transform(X_test))

In [14]:
y_test1[:4]

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 2.]])

In [15]:
ans1 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test1.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans1.index.name = 'id'
ans1.to_csv('ans1.csv')

In [16]:
!head ans1.csv

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,non functional
28965,non functional
36301,non functional


In [17]:
import collections

In [21]:
class HandyFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:,'amount_per_man'] = X.loc[:,'amount_tsh']/X.loc[:,'population']
        return X

class OutcomeFrequences(BaseException, TransformerMixin):
    def __init__(self, groupby):
        self.groupby = groupby
        self.cnts_ = None
        self.fq_cols_ = None
    def fit(self, X, y=None):
        possible_outcomes = [_.split()[-1] if isinstance(_, str) else _ for _ in np.unique(y)]
        n_outs = len(possible_outcomes)
        self.cnts_ = collections.defaultdict(lambda: np.zeros(n_outs))
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        for i in range(X.shape[0]):
            # assuming $y$ is label-encoded
            keys, out = tuple(X.iloc[i, igroupby].values), y[i]
            self.cnts_[keys][out] += 1 # no of `(keys, out)` occurences
        for k in self.cnts_:
            self.cnts_[k] /= self.cnts_[k].sum()
            
        self.possible_outcomes_ = possible_outcomes
        self.fq_cols_ = ['_'.join(self.groupby + [str(out), 'fq'],) for out in possible_outcomes]
        return self
    def transform(self, X):
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        new_cols = [None for _ in range(X.shape[0])]
        for i in range(X.shape[0]):
            keys = tuple(X.iloc[i, igroupby].values)
            if keys in self.cnts_:
                new_cols[i] = tuple([self.cnts_[keys][out] for out in self.possible_outcomes_])
        new_cols = pd.DataFrame(new_cols, columns=self.fq_cols_)
        for c in self.fq_cols_:
            new_cols.loc[:,c].fillna(new_cols.loc[:,c].median(), inplace=True)
        X = pd.concat((X, new_cols,), axis=1)
        return X

In [22]:
prepr2 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('handyfeats', HandyFeatures()),
    ('fq', OutcomeFrequences(['subvillage'])),
    ('drop', DropGarbage(cols=['id', 'recorded_by']))
])

y_enc = sklearn.preprocessing.LabelEncoder()
y = y_enc.fit_transform(y_train.iloc[:,1])
X = prepr2.fit_transform(X_train, y)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.66)


In [ ]:

!date
cbc2 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_te, y_te))
cbc2.fit(X_tr,
         y_tr,
         cat_features=np.where(np.array([dt == object for dt in X.dtypes]))[0],
         verbose=False)
!date
display('score: %s' % cbc2.score(X_te, y_te))

In [23]:

y_test2 = cbc2.predict(prepr2.transform(X_test))
ans2 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test2.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans2.index.name = 'id'
ans2.to_csv('ans2.csv')

TypeError: object of type 'NoneType' has no len()

In [ ]:
import sklearn.utils
import itertools

Whoa! I overfited again! Drivendata scored it $0.8102$.
We need some 1) diversification and 2) to validate model more carefully.
We'll split train set into three parts this time and ensure we've got equal proportions of all classes in all of the (new_train, cv, test) sets.

In [ ]:
def split_train_cv_test(X, y, proportions=(.6, .1, .3)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    # parts = []
    # print(len(xparts), 'parts')
    # for xpart, ypart in zip(xparts, yparts):
    #     parts.append(xpart)
    #     parts.append(ypart)
    parts = xparts + yparts
    return parts

In [ ]:
class OutcomeFrequences2(BaseException, TransformerMixin):
    def __init__(self, groupby, drop=False):
        self.groupby = groupby
        self.drop = drop
        self.cnts_ = None
        self.fq_cols_ = None
        
    def fit(self, X, y=None):
        possible_outcomes = [_.split()[-1] if isinstance(_, str) else _ for _ in np.unique(y)]
        n_outs = len(possible_outcomes)
        self.cnts_ = collections.defaultdict(lambda: np.zeros(n_outs))
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        for i in range(X.shape[0]):
            # assuming $y$ is label-encoded
            keys, out = tuple(X.iloc[i, igroupby].values), y[i]
            self.cnts_[keys][out] += 1 # no of `(keys, out)` occurences
        for k in self.cnts_:
            self.cnts_[k] //= self.cnts_[k].sum()
            
        self.possible_outcomes_ = possible_outcomes
        self.fq_cols_ = ['_'.join(self.groupby + [str(out), 'fq'],) for out in possible_outcomes]
        return self
    
    def transform(self, X):
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        new_cols = [[None for out in self.possible_outcomes_] for _ in range(X.shape[0])]
        for i in range(X.shape[0]):
            keys = tuple(X.iloc[i, igroupby].values)
            if keys in self.cnts_:
                new_cols[i] = tuple([self.cnts_[keys][out] for out in self.possible_outcomes_])
        # print(str(X)[:100])
        # print(str(new_cols)[:100])
        # print(str(self.fq_cols_[:100]))
        new_cols = pd.DataFrame(new_cols, columns=self.fq_cols_)
        for c in self.fq_cols_:
            new_cols.loc[:,c].fillna(new_cols.loc[:,c].median(), inplace=True)
        X = pd.concat((X, new_cols,), axis=1)
        if self.drop:
            X.drop(self.groupby, axis=1, inplace=True)
        return X

class ExplicitAge(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:, 'age'] = X.loc[:,'date_recorded_year'] - X.loc[:,'construction_year']

prepr3 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('handyfeats', HandyFeatures()),
] + [
    (catcol + '_fq', OutcomeFrequences2(catcol, drop=True))
] + [
    ('age', ExplicitAge()),
    ('drop', DropGarbage(cols=[
        'id', 'recorded_by',
        'region', 'funder', 'installer',
        'wtp_name'
    ]))
])

y_enc = sklearn.preprocessing.LabelEncoder()
y = y_enc.fit_transform(y_train.iloc[:,1])
X = prepr3.fit_transform(X_train, y)

In [ ]:
X_tr, X_cv, X_te, y_tr, y_cv, y_te = split_train_cv_test(X, y)

In [ ]:
!date
cbc3 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_cv, y_cv))
cbc3.fit(X_tr,
         y_tr,
         cat_features=np.where(np.array([dt == object for dt in X.dtypes]))[0],
         verbose=True)
!date
display('score: %s' % cbc3.score(X_te, y_te))

In [ ]:
y_test3 = cbc3.predict(prepr3.transform(X_test))

In [ ]:
ans3 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test3.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans3.index.name = 'id'
ans3.to_csv('ans3.csv')

In [ ]:
!head ans1.csv ans2.csv ans3.csv

In [ ]:
display('score on a full train: %s' % cbc3.score(X, y))

In [ ]:
prepr4 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('handyfeats', HandyFeatures()),
    ('district_fq', OutcomeFrequences2(['district_code'])),
    ('basin_fq', OutcomeFrequences2(['basin'])),
    ('installer_fq', OutcomeFrequences2(['installer'])),
    ('extraction_t_c_fq', OutcomeFrequences2(['extraction_type_class'])),
    ('wpt_type_fq', OutcomeFrequences2(['waterpoint_type'])),
    ('wpt_name_fq', OutcomeFrequences2(['wtp_name'])),
    ('age', ExplicitAge()),
    ('drop', DropGarbage(cols=[
        'id', 'recorded_by',
        'region', 'funder', 'installer',
        'wtp_name', 'scheme_management',
        'num_private', 'subvillage', 
        'longtitude', 'latitude',
        'date_recorded_day', 'date_recorded_weekday'
    ]))
])

y = y_enc.fit_transform(y_train.iloc[:,1])
X = prepr4.fit_transform(X_train, y)
X_tr, X_cv, X_te, y_tr, y_cv, y_te = split_train_cv_test(X, y)
!date
cbc4 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_cv, y_cv))
cat_features = np.where(np.array([(X[c].dtype == object) or ('code' in c) for dt in X.columns.values]))[0]
cbc4.fit(X_tr,
         y_tr,
         cat_features=cat_features,
         verbose=True)
!date
display('score: %s' % cbc4.score(X_te, y_te))
y_test4 = cbc4.predict(prepr4.transform(X_test))
ans4 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test4.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans4.index.name = 'id'
ans4.to_csv('ans4.csv')